In [349]:
import os
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # to avoid too many logging messages
import pandas as pd
import numpy as np
import random
import shutil
import tensorflow as tf, re, math
import tensorflow.keras.backend as K
import sklearn
import matplotlib.pyplot as plt
import tensorflow_addons as tfa
import tensorflow_probability as tfp
import wandb
import yaml

from IPython import display as ipd
from glob import glob
from tqdm import tqdm
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import roc_auc_score
from sklearn.utils.class_weight import compute_class_weight

from keras_cv_attention_models import efficientnet

In [350]:
class CFG:

    model_name = 'EfficientNetV2B0'

    batch_size = 32
    epochs = 5

    seed = 123

    folds = 5

    img_size = [512, 512]

    target_col  = [ "bowel_injury", "extravasation_injury", "kidney_healthy", "kidney_low",
                   "kidney_high", "liver_healthy", "liver_low", "liver_high",
                   "spleen_healthy", "spleen_low", "spleen_high"]

In [351]:
def seeding(SEED):
    np.random.seed(SEED)
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    tf.random.set_seed(SEED)
    print('seeding done!!!')

seeding(CFG.seed)

seeding done!!!


In [352]:
df = pd.read_csv('./Dataset/train.csv')
test_df = pd.read_csv('./Dataset/test.csv')

In [353]:
print(df.shape)
print(test_df.shape)

(13015, 21)
(3, 4)


In [354]:
df['stratify'] = ''

for col in CFG.target_col:
    df['stratify'] += df[col].astype(str)

df = df.reset_index(drop=True)

sgkf = StratifiedGroupKFold(n_splits=CFG.folds, shuffle=True, random_state=CFG.seed)

for  fold , (train_idx, val_idx) in enumerate(sgkf.split(df, df['stratify'], df['patient_id'])):
    df.loc[val_idx, 'fold'] = fold

/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/sklearn/model_selection/_split.py:950: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [355]:
def build_decoder():


    def decode_image(path):

        file_binary = tf.io.read_file(path)
        image = tf.image.decode_png(file_binary, channels=3, dtype=tf.uint8)
        image = tf.image.resize(image, CFG.img_size, method='bilinear')
        image = tf.cast(image, tf.float32) / 255.0
        image = tf.reshape(image, [*CFG.img_size, 3])

        return image
    

    def decode_label(label):
        
        label = tf.cast(label, tf.float32)
        bowel = label[0:1]
        extra = label[1:2]
        liver = label[2:5]
        kidney = label[5:8]
        spleen = label[8:11]
        
        return bowel, extra, liver, kidney, spleen
    
    def decode_with_labels(path, label):

        return decode_image(path), decode_label(label)

    return decode_with_labels
    


def build_dataset(paths, labels, batch_size=32, drop_remainder=False):

    slices = (paths, labels)

    ds = tf.data.Dataset.from_tensor_slices(slices)

    ds = ds.map(build_decoder())

    ds = ds.batch(batch_size, drop_remainder=drop_remainder)

    return ds

In [356]:
def build_model(model_name=CFG.model_name, dim=CFG.img_size):


        # backbone
        base = getattr(efficientnet, model_name)(input_shape=(*dim,3), pretrained='imagenet', num_classes=0)

        input = base.inputs

        x = base.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)

        # necks
        x_bowel = tf.keras.layers.Dense(32, activation='silu')(x)
        x_extra = tf.keras.layers.Dense(32, activation='silu')(x)
        x_liver = tf.keras.layers.Dense(32, activation='silu')(x)
        x_kidney = tf.keras.layers.Dense(32, activation='silu')(x)
        x_spleen = tf.keras.layers.Dense(32, activation='silu')(x)

        # heads
        out_bowel = tf.keras.layers.Dense(1, name='bowel', activation='sigmoid')(x_bowel)
        out_extra = tf.keras.layers.Dense(1, name='extra', activation='sigmoid')(x_extra)
        out_liver = tf.keras.layers.Dense(3, name='liver', activation='softmax')(x_liver)
        out_kidney = tf.keras.layers.Dense(3, name='kidney', activation='softmax')(x_kidney)
        out_spleen = tf.keras.layers.Dense(3, name='spleen', activation='softmax')(x_spleen)

        out_combined = tf.keras.layers.Concatenate()([out_bowel, out_extra])

        output = [out_combined, out_liver, out_kidney, out_spleen]

        model = tf.keras.Model(inputs=input, outputs=output)


        optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.0001)

        loss = {'bowel':tf.keras.losses.BinaryCrossentropy(),
                'extra':tf.keras.losses.BinaryCrossentropy(),
                'liver':tf.keras.losses.CategoricalCrossentropy(),
                'kidney':tf.keras.losses.CategoricalCrossentropy(),
                'spleen':tf.keras.losses.CategoricalCrossentropy()}

        metrics = {'bowel':['accuracy'],
                'extra':['accuracy'],
                'liver':['accuracy'],
                'kidney':['accuracy'],
                'spleen':['accuracy']}

        model.compile(optimizer=optimizer,
                loss=loss,
                metrics=[metrics])

        return model


In [357]:

for fold in np.arange(CFG.folds):

    train_df = df.query("fold!=@fold")
    valid_df = df.query("fold==@fold")

    train_paths  = train_df.image_path.values
    train_labels = train_df[CFG.target_col].values.astype(np.float32)
    valid_paths  = valid_df.image_path.values
    valid_labels = valid_df[CFG.target_col].values.astype(np.float32)
    test_paths   = test_df.image_path.values

    K.clear_session()

    model = build_model(CFG.model_name, dim=CFG.img_size)

    train_ds = build_dataset(train_paths, train_labels)
    valid_ds = build_dataset(valid_paths, valid_labels)

    history = model.fit(train_ds, validation_data = valid_ds, epochs=CFG.epochs)
                        
    best_epoch = np.argmin(history.history['val_loss'])
    best_loss = history.history['val_loss'][best_epoch]
    best_acc_bowel = history.history['val_bowel_accuracy'][best_epoch]
    best_acc_extra = history.history['val_extra_accuracy'][best_epoch]
    best_acc_liver = history.history['val_liver_accuracy'][best_epoch]
    best_acc_kidney = history.history['val_kidney_accuracy'][best_epoch]
    best_acc_spleen = history.history['val_spleen_accuracy'][best_epoch]

    best_acc = np.mean([best_acc_bowel, best_acc_extra, best_acc_liver, best_acc_kidney, best_acc_spleen])
    print(f'FOLD {fold} RESULTS')
    print(f'BEST Loss  : {best_loss:.3f}\nBEST Acc   : {best_acc:.3f}\nBEST Epoch : {best_epoch}\n')
    print(f'Bowel : {best_acc_bowel:.3f}')
    print(f'Extravasation : {best_acc_extra:.3f}')
    print(f'Liver: {best_acc_liver:.3f}')
    print(f'Kidney: {best_acc_kidney:.3f}')
    print(f'Spleen: {best_acc_spleen:.3f}')


>>>> Load pretrained from: /Users/makoto/.keras/models/efficientnetv2-b0-imagenet.h5
Epoch 1/5


ValueError: in user code:

    File "/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/keras/src/engine/training.py", line 1338, in train_function  *
        return step_function(self, iterator)
    File "/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/keras/src/engine/training.py", line 1322, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/keras/src/engine/training.py", line 1303, in run_step  **
        outputs = model.train_step(data)
    File "/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/keras/src/engine/training.py", line 1085, in train_step
        return self.compute_metrics(x, y, y_pred, sample_weight)
    File "/Users/makoto/homebrew/Caskroom/miniforge/base/envs/rsna2023/lib/python3.10/site-packages/keras/src/

In [ ]:
train_ds

<_BatchDataset element_spec=(TensorSpec(shape=(None, 512, 512, 3), dtype=tf.float32, name=None), (TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)))>

In [ ]:
valid_df

,patient_id,bowel_healthy,bowel_injury,extravasation_healthy,extravasation_injury,kidney_healthy,kidney_low,kidney_high,liver_healthy,liver_low,...,spleen_high,any_injury,series_id,instance_number,injury_name,image_path,width,height,stratify,fold
128,10217,1,0,0,1,1,0,0,0,1,...,1,1,16066,256,Active_Extravasation,./Dataset/train_images/10217/16066/256.png,512,512,01100010001,0.0
129,10217,1,0,0,1,1,0,0,0,1,...,1,1,16066,257,Active_Extravasation,./Dataset/train_images/10217/16066/257.png,512,512,01100010001,0.0
130,10217,1,0,0,1,1,0,0,0,1,...,1,1,16066,258,Active_Extravasation,./Dataset/train_images/10217/16066/258.png,512,512,01100010001,0.0
131,10217,1,0,0,1,1,0,0,0,1,...,1,1,16066,259,Active_Extravasation,./Dataset/train_images/10217/16066/259.png,512,512,01100010001,0.0
132,10217,1,0,0,1,1,0,0,0,1,...,1,1,16066,260,Active_Extravasation,./Dataset/train_images/10217/16066/260.png,512,512,01100010001,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12831,7642,0,1,1,0,1,0,0,0,1,...,0,1,778,514,Bowel,./Dataset/train_images/7642/778/514.png,512,512,10100010010,0.0
12832,7642,0,1,1,0,1,0,0,0,1,...,0,1,778,515,Bowel,./Dataset/train_images/7642/778/515.png,512,512,10100010010,0.0
12833,7642,0,1,1,0,1,0,0,0,1,...,0,1,778,516,Bowel,./Dataset/train_images/7642/778/516.png,512,512,10100010010,0.0
12834,7642,0,1,1,0,1,0,0,0,1,...,0,1,778,517,Bowel,./Dataset/train_images/7642/778/517.png,512,512,10100010010,0.0
